In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import cv2
import pickle
from skimage.feature import hog
from skimage.color import rgb2gray
from tqdm import tqdm

PATH = "/content/drive/MyDrive/C-NMC_Leukemia"

#add training data
train_dataset_0_all = glob.glob(PATH + "/training_data/fold_0/all/*.bmp")
train_dataset_0_hem = glob.glob(PATH + "/training_data/fold_0/hem/*.bmp")
train_dataset_1_all = glob.glob(PATH + "/training_data/fold_1/all/*.bmp")
train_dataset_1_hem = glob.glob(PATH + "/training_data/fold_1/hem/*.bmp")
train_dataset_2_all = glob.glob(PATH + "/training_data/fold_2/all/*.bmp")
train_dataset_2_hem = glob.glob(PATH + "/training_data/fold_2/hem/*.bmp")
# print(len(train_dataset_0_all))

#include validation data
valid_data=pd.read_csv(PATH + "/validation_data/C-NMC_test_prelim_phase_data_labels.csv") #reading csv

av = valid_data[valid_data["labels"] == 1] # av-> all validation
hv = valid_data[valid_data["labels"] == 0] # hv -> hem validation

VAL_PATH = PATH + "/validation_data/C-NMC_test_prelim_phase_data/"
AVL = [VAL_PATH +  i for i in list(av.new_names)]
HVL = [VAL_PATH +  i for i in list(hv.new_names)]

#merge training and validation data
A = list()
H = list()
A.extend(train_dataset_0_all)
A.extend(train_dataset_1_all)
A.extend(train_dataset_2_all)
A.extend(AVL)
H.extend(train_dataset_0_hem)
H.extend(train_dataset_1_hem)
H.extend(train_dataset_2_hem)
H.extend(HVL)

print("The number of Acute lymphoblastic leukemia samples: ", len(A))
print("The number of samples without leukemia :",len(H))

#labling
Label_A = [1]*len(A)
Label_H = [0]*len(H)
Label_A.extend(Label_H)

#dataframing
A.extend(H)

dataset = []

The number of Acute lymphoblastic leukemia samples:  8491
The number of samples without leukemia : 4049


# **ANN preprocessing functions**

In [ ]:
def ANN_img_preprocessing(img):
  image = cv2.imread(img)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  resized_gray = cv2.resize(gray, (120,120))
  edged_gray = cv2.Canny(resized_gray, 100, 200)
  edged_gray = np.array(edged_gray).flatten()
  return edged_gray

In [ ]:
for i in tqdm(range(0,len(A))):
    dataset.append(ANN_img_preprocessing(A[i]))

In [ ]:
pickle_out = open("/content/drive/MyDrive/ML_Results/Preprocessing/X_ANN_Img_Pre.pickle","wb")
pickle.dump(dataset, pickle_out)
pickle_out.close()

pickle_out = open("/content/drive/MyDrive/ML_Results/Preprocessing/Y_ANN_Img_Pre.pickle","wb")
pickle.dump(Label_A, pickle_out)
pickle_out.close()

# **SVM preprocessing functions**

In [ ]:
def SVM_img_preprocessing(img):
  image = cv2.imread(img)
  gray = rgb2gray(image)
  resized_gray = cv2.resize(gray, (400,400))
  hog_features = hog(resized_gray,block_norm='L2-Hys',pixels_per_cell=(16,16))
  features = np.array(hog_features).flatten()
  return features

In [ ]:
for i in tqdm(range(0,len(A))):
    dataset.append(SVM_img_preprocessing(A[i]))

100%|██████████| 12540/12540 [10:08<00:00, 20.61it/s]


In [ ]:
pickle_out = open("/content/drive/MyDrive/ML_Results_Main/X_SVM_Img_Pre.pickle","wb")
pickle.dump(dataset, pickle_out)
pickle_out.close()

pickle_out = open("/content/drive/MyDrive/ML_Results_Main/Preprocessing/Y_SVM_Img_Pre.pickle","wb")
pickle.dump(Label_A, pickle_out)
pickle_out.close()